In [1]:
# 📦 Importation des bibliothèques  
from sentence_transformers import SentenceTransformer  
from sklearn.neighbors import NearestNeighbors  
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
import pickle  
import os  
import warnings  
warnings.filterwarnings("ignore")  

In [2]:
# 📊 Chargement des données préparées  
print("Chargement des données préparées...")  
try:  
    # Essayer de charger depuis le fichier pickle  
    with open('../data/processed/clean_dataset.pkl', 'rb') as f:  
        df = pickle.load(f)  
    print("Données chargées depuis le fichier pickle.")  
except:  
    # Sinon, charger depuis le CSV  
    df = pd.read_csv('../data/processed/clean_dataset.csv')  
    print("Données chargées depuis le fichier CSV.")  
  
print(f"Forme du dataset: {df.shape}")  
print("\nAperçu des données:")  
display(df.head())  

Chargement des données préparées...
Données chargées depuis le fichier pickle.
Forme du dataset: (5000, 14)

Aperçu des données:


Uniq Id                        Product Id  \
0  1705736792d82aa2f2d3caf1c07c53f4  2e17bf4acecdece67fc00f07ad62c910   
1  95a9fe6f4810fcfc7ff244fd06784f11  076e5854a62dd283c253d6bae415af1f   
2  8d4d0330178d3ed181b15a4102b287f2  8a4fe5d9c7a6ed26cc44d785a454b124   
3  fddc4df45b35efd886794b261f730c51  03b5fb878a33eadff8b033419eab9669   
4  0990cf89a59ca6a0460349a3e4f51d42  ce3d761e57d6ccad80619297b5b1bcbc   

   Product Rating  Product Reviews Count  Clicked  View_Time_Sec  \
0        4.218750             571.035565        0              0   
1        3.966667             571.035565        1            147   
2        4.500000           29221.000000        0              0   
3        4.024138             571.035565        0              0   
4        5.000000             131.000000        1            177   

                                        Product Name  \
0  OPI Infinite Shine, Nail Lacquer Nail Polish, ...   
1  Nice n Easy Permanent Color, 111 Natural Mediu...   
2  Clairol Nice N Easy Permanent Color 7/106A Nat...   
3  Kokie Professional Matte Lipstick, Hot Berry, ...   
4  Gillette TRAC II Plus Razor Blade Refills, Fit...   

                                 Product Description  \
0                                                      
1  Pack of 3 Pack of 3 for the UPC: 381519000201 ...   
2  This Clairol Nice N Easy Permanent Color gives...   
3  Calling all matte lip lovers! Indulge in our r...   
4  In 1971, Gillette introduced the Trac II razor...   

                                        Product Tags  \
0  OPI Infinite Shine, Nail Lacquer Nail Polish, ...   
1  Nice 'n Easy Permanent Color, 111 Natural Medi...   
2  Clairol Nice 'N Easy Permanent Color 7/106A Na...   
3  Kokie Professional Matte Lipstick, Hot Berry, ...   
4  Gillette TRAC II Plus Razor Blade Refills, Fit...   

                                    Product Category    Product Brand  \
0  Premium Beauty > Premium Makeup > Premium Nail...              OPI   
1  Beauty > Hair Care > Hair Color > Auburn Hair ...      Nice'n Easy   
2  Beauty > Hair Care > Hair Color > Permanent Ha...          Clairol   
3                              Beauty > Makeup > Lip  Kokie Cosmetics   
4  Seasonal > Stock Up Essentials > Personal Care...         Gillette   

                                           Full_Text  Norm_View_Time    Rating  
0   opi infinite shine, nail lacquer nail polish,...        0.000000  2.109375  
1  pack of 3 pack of 3 for the upc: 381519000201 ...        0.816667  4.300000  
2  this clairol nice n easy permanent color gives...        0.000000  2.250000  
3  calling all matte lip lovers! indulge in our r...        0.000000  2.012069  
4  in 1971, gillette introduced the trac ii razor...        0.983333  4.983333

In [3]:
  
# 📊 Préparation des données pour le filtrage basé sur le contenu  
print("\n📊 Préparation des données pour le filtrage basé sur le contenu (BERT + KNN)...")  
  
# Création d'un DataFrame avec des produits uniques  
products_df = df.drop_duplicates('Product Id')  
print(f"Nombre de produits uniques: {products_df['Product Id'].nunique()}")  


📊 Préparation des données pour le filtrage basé sur le contenu (BERT + KNN)...
Nombre de produits uniques: 4802


In [4]:
 
# 🔧 Génération des embeddings BERT  
print("\n🔧 Génération des embeddings BERT...")  
# Chargement du modèle BERT pré-entraîné  
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  
print("Modèle BERT chargé: paraphrase-MiniLM-L6-v2")  
  
# Génération des embeddings pour chaque produit  
print("Génération des embeddings pour les descriptions de produits...")  
embeddings = model.encode(products_df['Full_Text'].tolist(), show_progress_bar=True)  
print(f"Forme des embeddings: {embeddings.shape}")  


🔧 Génération des embeddings BERT...


Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Modèle BERT chargé: paraphrase-MiniLM-L6-v2
Génération des embeddings pour les descriptions de produits...


Batches:   0%|          | 0/151 [00:00<?, ?it/s]

Forme des embeddings: (4802, 384)


In [5]:
  
# 🔍 Modèle KNN pour BERT  
print("\n🔍 Entraînement du modèle KNN sur les embeddings BERT...")  
knn_model = NearestNeighbors(  
    n_neighbors=10,       # Nombre de voisins à trouver  
    algorithm='auto',     # Choix automatique de l'algorithme  
    metric='cosine'       # Mesure de similarité cosinus  
)  
knn_model.fit(embeddings)  
print("Modèle KNN entraîné avec succès.")  


🔍 Entraînement du modèle KNN sur les embeddings BERT...
Modèle KNN entraîné avec succès.


In [6]:
  
# 🔮 Exemple de recommandation  
print("\n🔮 Exemple de recommandation avec BERT + KNN:")  
# Sélectionner un produit aléatoire  
sample_idx = np.random.randint(0, len(products_df))  
sample_product = products_df.iloc[sample_idx]  
print(f"Produit sélectionné: {sample_product['Product Name']}")  
print(f"Catégorie: {sample_product['Product Category']}")  
  
# Trouver les produits similaires  
sample_vector = embeddings[sample_idx].reshape(1, -1)  
distances, indices = knn_model.kneighbors(sample_vector)  
  
print("\nProduits similaires:")  
for i, idx in enumerate(indices[0][1:6]):  # Ignorer le premier qui est le produit lui-même  
    similar_product = products_df.iloc[idx]  
    print(f"{i+1}. {similar_product['Product Name']} (Similarité: {1-distances[0][i+1]:.4f})")  
    print(f"   Catégorie: {similar_product['Product Category']}")  


🔮 Exemple de recommandation avec BERT + KNN:
Produit sélectionné: GloMinerals Eye Blender (Brush)
Catégorie: Premium Beauty > Premium Tools & Brushes > Premium Makeup Tools & Brushes

Produits similaires:
1. NARS Eye Paint, Baalbek (Similarité: 0.6871)
   Catégorie: Premium Beauty > Premium Makeup > Premium Eyes > Premium Eyeshadow
2. Obagi Elastiderm Eye Cream, 0.5 oz (Similarité: 0.6861)
   Catégorie: Premium Beauty > Premium Facial Skincare > Premium Eye Cream & Treatments
3. Clinique All About Eyes, Eye Cream, 0.5 Oz (Similarité: 0.6691)
   Catégorie: Premium Beauty > Premium Facial Skincare > Premium Eye Cream & Treatments
4. Power Oils Lip Gloss (Similarité: 0.6657)
   Catégorie: Premium Beauty > Premium Makeup > Premium Lips > Premium Lip Gloss
5. Laura Mercier Tone Perfecting Eye Gel Crème 0.5 oz (Similarité: 0.6629)
   Catégorie: Premium Beauty > Premium Facial Skincare > Premium Eye Cream & Treatments


In [8]:
# 📊 Évaluation  
print("\n📊 Évaluation du modèle BERT + KNN:")  
# Calcul de la précision de la catégorie  
same_category_count = 0  
total_recommendations = 0  
  
# Échantillonner 100 produits aléatoires pour l'évaluation  
sample_size = min(100, len(products_df))  
sample_indices = np.random.choice(len(products_df), sample_size, replace=False)  
  
for idx in sample_indices:  
    product = products_df.iloc[idx]  
    product_vector = embeddings[idx].reshape(1, -1)  
    distances, indices = knn_model.kneighbors(product_vector)  
      
    # Vérifier si les 5 premiers produits recommandés sont de la même catégorie  
    for rec_idx in indices[0][1:6]:  
        recommended_product = products_df.iloc[rec_idx]  
        if recommended_product['Product Category'] == product['Product Category']:  
            same_category_count += 1  
        total_recommendations += 1  
  
category_precision = same_category_count / total_recommendations  
print(f"Précision de la catégorie: {category_precision:.4f}") 


📊 Évaluation du modèle BERT + KNN:
Précision de la catégorie: 0.2700


In [9]:
  
# Sauvegarde des résultats pour l'évaluation comparative  
results = {  
    'bert_knn': {  
        'category_precision': category_precision  
    }  
}  
  
# 💾 Sauvegarde du modèle et des résultats  
print("\n💾 Sauvegarde du modèle et des résultats...")  
  
# Créer les dossiers s'ils n'existent pas  
os.makedirs('../models/content_based', exist_ok=True)  
os.makedirs('../results', exist_ok=True)  
  
# Sauvegarde des embeddings BERT  
with open('../models/content_based/bert_embeddings.pkl', 'wb') as f:  
    pickle.dump(embeddings, f)  
print("Embeddings BERT sauvegardés dans '../models/content_based/bert_embeddings.pkl'")  
  
# Sauvegarde du modèle KNN  
with open('../models/content_based/bert_knn_model.pkl', 'wb') as f:  
    pickle.dump(knn_model, f)  
print("Modèle KNN pour BERT sauvegardé dans '../models/content_based/bert_knn_model.pkl'")  
  
# Sauvegarde des résultats  
with open('../results/bert_results.pkl', 'wb') as f:  
    pickle.dump(results, f)  
print("Résultats sauvegardés dans '../results/bert_results.pkl'")  
  
print("\n✅ Filtrage basé sur le contenu (BERT + KNN) terminé!")


💾 Sauvegarde du modèle et des résultats...
Embeddings BERT sauvegardés dans '../models/content_based/bert_embeddings.pkl'
Modèle KNN pour BERT sauvegardé dans '../models/content_based/bert_knn_model.pkl'
Résultats sauvegardés dans '../results/bert_results.pkl'

✅ Filtrage basé sur le contenu (BERT + KNN) terminé!
